In [1]:
!pip install feature-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 2.5 MB/s eta 0:00:00


# Hiper-parâmetros GridSearch de ML -  Classificação

Este notebook irá apresentar os principais hiperparâmetros e seus respectivos valores que podemos testar em um GridSearch.

## Carregar os dados

In [2]:
import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from sklearn.model_selection import train_test_split
import pandas as pd
import os

WORK_DIR = '/content/drive/My Drive/datasets'
DATA_DIR = os.path.join(WORK_DIR, 'olist')
df_abt = pd.read_csv(os.path.join(DATA_DIR, 'propensao_revenda_abt.csv'))

# pega a base de treinamento
df_train = df_abt.query('data_ref_safra < "2018-03-01"')

# pega a base de avaliação (out of time)
df_oot   = df_abt.query('data_ref_safra == "2018-03-01"')

key_vars = ['data_ref_safra', 'seller_id']
num_vars = ['tot_orders_12m', 'tot_items_12m', 'tot_items_dist_12m', 'receita_12m', 'recencia']
cat_vars = ['uf']
target = 'nao_revendeu_next_6m'

features = cat_vars + num_vars

# dados de treinamento
X_train = df_train[features]
y_train = df_train[target]

# dados de avaliação (out of time)
X_oot = df_oot[features]
y_oot = df_oot[target]



In [5]:

from sklearn.pipeline import Pipeline
from feature_engine.imputation import ArbitraryNumberImputer
from feature_engine.imputation import CategoricalImputer
from feature_engine.encoding import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier


datapipe = [
            ('numeric_imputer', ArbitraryNumberImputer(variables=num_vars, arbitrary_number=-999)),
            ('categoric_imputer', CategoricalImputer(variables=cat_vars, return_object=True)),
            ('one_hot_encoder', OneHotEncoder(variables=cat_vars))
]


## Decision Tree

Os principais hiperparâmetros que podemos utilizar no GridSearch:

* `max_depth`: Profundidade da árvore. O valor padrão será construir a árvore até que as folhas contenham menos que o valor definido em `min_samples_split`. Possíveis valores: `2, 3, 4, 5, 6, 7, ... `.
* `criterion`: Função de separação da árvore. Valores possíveis: `gini` e `entropy`.
* `class_weight`: pesos das classes em um dicionário. O valor `balanced` definirá valores na proporção inversa das frequências das classes. Possíveis valores: `balanced` e `None`.
* `min_samples_split`: Número mínimo de amostras exigido para separar um nó. Valor padrão é 2.

In [6]:
t1 = time.time()

from sklearn.model_selection import GridSearchCV

pipeline = Pipeline(steps=datapipe + [('decision_tree', DecisionTreeClassifier())])

parametros = {
  'decision_tree__max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
  'decision_tree__criterion': ['gini', 'entropy'],
  'decision_tree__class_weight': ['balanced', None],
  'decision_tree__min_samples_split': [2],
}

grid_search = GridSearchCV(pipeline, parametros, scoring='roc_auc', cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print()
print('='*100)
print(grid_search.best_params_)
print('\nDemorou {} segundos para GridSearch para Decision Tree'.format(time.time() - t1))
print('='*100)
print()

Fitting 5 folds for each of 40 candidates, totalling 200 fits

{'decision_tree__class_weight': None, 'decision_tree__criterion': 'entropy', 'decision_tree__max_depth': 4, 'decision_tree__min_samples_split': 2}

Demorou 31.705455780029297 segundos para GridSearch para Decision Tree




## Random Forest

Os principais hiperparâmetros que podemos utilizar no GridSearch:

* `n_estimators`: Número de árvores na floresta aleatório. Possíveis valores: `120, 300, 500, 800, 1200`.
* `max_depth`: Profundidade da árvore. O valor padrão será construir a árvore até que as folhas contenham menos que o valor definido em `min_samples_split`. Possíveis valores: `5, 8, 15, 25, 30, None`.
* `max_features`: Número de atributos (características) para analisar na separação. O padrão são todos (`auto`).
Possíveis valores: `log2, sqrt, None`.
* `min_samples_split`: Número mínimo de amostras exigido para separar um nó. Valor padrão é 2. Possíveis valores: `2, 5, 10, 15, 100`.

* `min_samples_leaf`: Número mínimo de amostras em cada folha. Valor padrão é 1. Possíveis valores: `1, 2, 5, 10`.

* `class_weight`: pesos das classes em um dicionário. O valor `balanced` definirá valores na proporção inversa das frequências das classes. Possíveis valores: `balanced` e `None`.




In [7]:
t2 = time.time()

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

pipeline = Pipeline(steps=datapipe + [('random_forest', RandomForestClassifier())])

parametros = {
    'random_forest__n_estimators': [120, 300], # 500, 800, 1200],
    'random_forest__max_depth': [5, 8, 15], # 25, 30, None],
    'random_forest__max_features': ['log2'], # 'sqrt', 'None'],
    'random_forest__min_samples_split': [2, 5], # 10, 15, 100],
    'random_forest__min_samples_leaf': [1, 2], # 5, 10],
    'random_forest__class_weight': ['balanced', None]
}

grid_search = GridSearchCV(pipeline, parametros, scoring='roc_auc', cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print()
print('='*100)
print(grid_search.best_params_)
print('\nDemorou {} segundos para GridSearch para Random Forest'.format(time.time() - t2))
print('='*100)
print()

Fitting 5 folds for each of 48 candidates, totalling 240 fits

{'random_forest__class_weight': None, 'random_forest__max_depth': 15, 'random_forest__max_features': 'log2', 'random_forest__min_samples_leaf': 1, 'random_forest__min_samples_split': 2, 'random_forest__n_estimators': 300}

Demorou 168.29576230049133 segundos para GridSearch para Random Forest




## XGBoost

Os principais hiperparâmetros que podemos utilizar no GridSearch:

* `learning_rate`: Taxa de aprendizagem (também conhecido como eta) para boosting (entre 0 e 1). Após cada passo de boosting (melhoria), os pesos recém adicionados são escalados de acordo com esse fator. Quanto menor o valor, mais conservador será, mas também serão necessárias mais árvores para convergir. Possíveis valores: `0.01, 0.015, 0.025, 0.05, 0.1`.
* `n_estimators`: Número de árvores na floresta aleatório. Possíveis valores: `500, 1000, 2000, 5000, 10000`.
* `max_depth`: Profundidade da árvore. O valor padrão será construir a árvore até que as folhas contenham menos que o valor definido em `min_samples_split`. Possíveis valores: `3, 5, 7, 9, 12, 15, 17, 25, None`.

* `colsample_bytree`: Fração das colunas a serem usadas por rodada. Possíveis valores: `0.6, 0.7, 0.8, 0.9, 1.0`.
* `subsample`: Fração das amostras a serem usadas na próxima rodada. Possíveis valores: `0.6, 0.7, 0.8, 0.9, 1.0`.

* `class_weight`: pesos das classes em um dicionário. O valor `balanced` definirá valores na proporção inversa das frequências das classes. Possíveis valores: `balanced` e `None`.




In [8]:
!pip install xgboost

In [9]:
t3 = time.time()
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline(steps=datapipe + [('xgboost', XGBClassifier())])

parametros = {
    'xgboost__learning_rate': [0.01, ], #[0.1, 0.01, 0.015, 0.025, 0.05, 0.1],
    'xgboost__n_estimators': [500], # [500, 1000, 2000, 5000, 10000],
    'xgboost__max_depth': [3], # [3, 5, 7, 9, 12, 15, 17, 25],
    'xgboost__colsample_bytree': [0.6], # [0.6, 0.7, 0.8, 0.9, 1.0],
    'xgboost__subsample': [0.6], # [0.6, 0.7, 0.8, 0.9, 1.0],
    'xgboost__class_weight': ['balanced', None],
}

grid_search = GridSearchCV(pipeline, parametros, scoring='roc_auc', cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print()
print('='*100)
print(grid_search.best_params_)
print('\nDemorou {} segundos para GridSearch para XGBoost.'.format(time.time() - t3))
print('='*100)
print()

Fitting 5 folds for each of 2 candidates, totalling 10 fits


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [16:51:02] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)



{'xgboost__class_weight': 'balanced', 'xgboost__colsample_bytree': 0.6, 'xgboost__learning_rate': 0.01, 'xgboost__max_depth': 3, 'xgboost__n_estimators': 500, 'xgboost__subsample': 0.6}

Demorou 4.368243932723999 segundos para GridSearch para XGBoost.



## LightGBM

Os principais hiperparâmetros que podemos utilizar no GridSearch:

* `learning_rate`: Taxa de aprendizagem (também conhecido como eta) para boosting (entre 0 e 1). Após cada passo de boosting (melhoria), os pesos recém adicionados são escalados de acordo com esse fator. Quanto menor o valor, mais conservador será, mas também serão necessárias mais árvores para convergir. Possíveis valores: `0.01, 0.015, 0.025, 0.05, 0.1`.
* `n_estimators`: Número de árvores na floresta aleatório. Possíveis valores: `500, 1000, 2000, 5000, 10000`.
* `max_depth`: Profundidade da árvore. O valor padrão será construir a árvore até que as folhas contenham menos que o valor definido em `min_samples_split`. Possíveis valores: `3, 5, 7, 9, 12, 15, 17, 25, None`.

* `colsample_bytree`: Fração das colunas a serem usadas por rodada. Possíveis valores: `0.6, 0.7, 0.8, 0.9, 1.0`.
* `subsample`: Fração das amostras a serem usadas na próxima rodada. Possíveis valores: `0.6, 0.7, 0.8, 0.9, 1.0`.

* `class_weight`: pesos das classes em um dicionário. O valor `balanced` definirá valores na proporção inversa das frequências das classes. Possíveis valores: `balanced` e `None`.

In [10]:
!pip install lightgbm

In [11]:
t4 = time.time()
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline(steps=datapipe + [('lgbm', LGBMClassifier())])

parametros = {
    'lgbm__learning_rate': [0.01, 0.05], # [0.01, 0.015, 0.025, 0.05, 0.1],
    'lgbm__n_estimators': [500], # [500, 1000, 2000, 5000, 10000],
    'lgbm__max_depth': [3], # [3, 5, 7, 9, 12, 15, 17, 25],
    'lgbm__colsample_bytree': [0.6], # [0.6, 0.7, 0.8, 0.9, 1.0],
    'lgbm__subsample': [0.8], # [0.6, 0.7, 0.8, 0.9, 1.0],
    'lgbm__class_weight': ['balanced', None],
}

grid_search = GridSearchCV(pipeline, parametros, scoring='roc_auc', cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print()
print('='*100)
print(grid_search.best_params_)
print('\nDemorou {} segundos para GridSearch para LGBM.'.format(time.time() - t4))
print('='*100)
print()

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[LightGBM] [Info] Number of positive: 1332, number of negative: 2163
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000279 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 908
[LightGBM] [Info] Number of data points in the train set: 3495, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.381116 -> initscore=-0.484815
[LightGBM] [Info] Start training from score -0.484815
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


## SVM

Os principais hiperparâmetros que podemos utilizar no GridSearch:

* `C`: Parâmetro de penalidade, quanto menor o valor, mais estreita será a fronteira de decisão (soft margin). Valor padrão 1.0. Possíveis valores: `0.001, 0.01, 0.1, 1, 10, 100, ...`
* `gamma`: Coeficiente utilizado para hiperplanos não lineares. Quanto maior o valor do gamma, maior será o ajuste nos dados. Isso pode levar ao overfitting Possíveis valores: `scale` e `auto`.
* `kernel`: Tipo do kernel: `linear`, `rbf` (padrão), `poly`, `sigmoid`, `precomputed` ou qualquer uma função.
* `class_weight`: pesos das classes em um dicionário. O valor `balanced` definirá valores na proporção inversa das frequências das classes. Possíveis valores: `balanced` e `None`.

In [12]:
t5 = time.time()
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from feature_engine.wrappers import SklearnTransformerWrapper


std_scaller = [('numeric_scaler', SklearnTransformerWrapper(variables=num_vars, transformer=StandardScaler()))]

pipeline = Pipeline(steps=datapipe + std_scaller + [('svc', SVC())])

parametros = {
    'svc__C': [0.01], # [0.001, 0.01, 0.1, 1, 10, 100],
    'svc__gamma': ['scale'], # ['scale', 'auto'],
    'svc__kernel': ['rbf', 'poly'], # ['rbf', 'poly', 'linear', 'sigmoid'],
    'svc__class_weight': ['balanced', None]
}
grid_search = GridSearchCV(pipeline, parametros, scoring='roc_auc', cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print()
print('='*100)
print(grid_search.best_params_)
print('\nDemorou {} segundos para GridSearch para SVC.'.format(time.time() - t5))
print('='*100)
print()

Fitting 5 folds for each of 4 candidates, totalling 20 fits

{'svc__C': 0.01, 'svc__class_weight': None, 'svc__gamma': 'scale', 'svc__kernel': 'poly'}

Demorou 21.124773263931274 segundos para GridSearch para SVC.



## Regressão Logística

Os principais hiperparâmetros que podemos utilizar no GridSearch:

* `C`: Ponto flutuante positivo. Inverso da força de regularização. Um valor menor significa uma regularização mais forte. Possíveis valores: `0.001, 0.01, 0.1, 1, 10, 100, ...`

* `penalty`: Norma de penalização `none` e `l2`.

* `class_weight`: pesos das classes em um dicionário. O valor `balanced` definirá valores na proporção inversa das frequências das classes. Possíveis valores: `balanced` e `None`.


* `fit_intercept`: Adiciona viés (bias) à função de decisão. Possíveis valores: True/False.


In [13]:
t6 = time.time()
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from feature_engine.wrappers import SklearnTransformerWrapper


std_scaller = [('numeric_scaler', SklearnTransformerWrapper(variables=num_vars, transformer=StandardScaler()))]

pipeline = Pipeline(steps=datapipe + std_scaller + [('logit', LogisticRegression())])

parametros = {
    'logit__penalty': ['none', 'l2'],
    'logit__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'logit__class_weight': ['balanced', None],
    'logit__fit_intercept': [True, False]
}
grid_search = GridSearchCV(pipeline, parametros, scoring='roc_auc', cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print()
print('='*100)
print(grid_search.best_params_)
print('\nDemorou {} segundos para GridSearch para Regressão Logistica.'.format(time.time() - t6))
print('='*100)
print()

Fitting 5 folds for each of 48 candidates, totalling 240 fits

{'logit__C': 0.01, 'logit__class_weight': 'balanced', 'logit__fit_intercept': False, 'logit__penalty': 'l2'}

Demorou 32.333207845687866 segundos para GridSearch para Regressão Logistica.

